# Sesión de Trabajo 5: ENTRENAMIENTO DE MODELOS DE INTELIGENCIA ARTIFICIAL (ANN)

**Asignatura:** Ciència de Dades i Intel·ligència Artificial Aplicades a la Construcció i Estructures  
**Institución:** ETSEIB - UPC  

---

## Objetivos de la sesión
La S6 comprende tanto el desarrollo de modelos predictivos de aprendizaje autónomo como la evaluación y extracción de conocimiento a partir de los mismos. Esta sesión se divide en 2 partes:

-La primera parte tiene como objetivo entrenar y evaluar el rendimiento de un modelo de clasificación multiclase para la predicción de la calificación elegida que comprenda lo mejor posible la relación entre los datos de entrada y el resultado de la certificación energética obtenida.Modelos elegidos:“NeuralNetwork”

-Una vez obtenido el mejor modelo posible, el objetivo de la segunda parte de la S6 es extraer conocimiento de la sostenibilidad del parque edificado catalán a partir del modelo entrenado.


* **S6.T0.** Separación de las variables de entrada (X) / salida (Y) y el set de entrenamiento/validación de la BBDD

* **S6.T1.** Configuración y entrenamiento de un clasificador basado en ANN

* **S6.T2.** Evaluación del rendimiento del modelo

* **S6.T3.** Interpretación del modelo

---



### S6.T0. Importar la base de datos y preparación de los datasets
* Conectamos con Google Drive.
* Cargamos el archivo `BBDD_ST5.csv`.
* Optimizamos los tipos de datos (`float32` e `int32`) para no saturar la RAM de Colab.
* Creación Dataset input/outputs
* División Train/Test (80 % entrenamiento, 20 % validación)

⏳ *Tiempo de ejecución estimado: 1-2 minutos (dependiendo de la velocidad de conexión a Drive).*


In [ ]:
# Importar librerías
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import warnings
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.exceptions import ConvergenceWarning
import shap

# Configuración
warnings.filterwarnings("ignore", category=ConvergenceWarning)
pd.set_option('future.no_silent_downcasting', True)

# Montar Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Ruta y Carga
ruta_drive = '/content/drive/MyDrive/Sostenibilidad/BBDD_ST5.csv'
df = pd.read_csv(ruta_drive, low_memory=False)

# Optimización

float64_cols = df.select_dtypes(include='float64').columns.tolist()
df[float64_cols] = df[float64_cols].astype('float32')
int64_cols = df.select_dtypes(include='int64').columns.tolist()
df[int64_cols] = df[int64_cols].astype('int32')

# División de datos
target = "Qualificació de consum d'energia primaria no renovable"
x_train, x_test, y_train, y_test = train_test_split(
    df.drop(target, axis=1), #Inputs
    df[target], #Output
    test_size=0.2, #Tamaño del set de validación (20%)
    random_state=1 # Semilla para controlar la aleatoriedad. No tocar
)

print(f"Base de datos cargada. Entrenamiento: {x_train.shape[0]} | Test: {x_test.shape[0]}")

---

### S6.T1. Configuración y entrenamiento de la Red Neuronal (ANN)

Entrena un modelo basado en “Neural Network” para predecir la calificación energética de un edificio (el output serán la calificación energética obtenida).

Podéis evaluar la influencia de los siguientes parámetros en la eficiencia del modelo:

* **Capas ocultas y neuronas por capa:** Define la profundidad del "cerebro". Ejemplo: (10, 20) son dos capas con 10 y 20 neuronas respectivamente.

* **Iteraciones**: Es el número de veces que el modelo revisa los datos (épocas). Si es muy bajo, no aprende; si es muy alto, tarda demasiado.

* **Tasa de aprendizaje (learning rate)**: Controla qué tan rápido cambia el modelo en respuesta al error.

>>-Muy alta: El modelo aprende rápido pero puede volverse inestable.

>>-Muy baja: El aprendizaje es muy lento y puede quedar atrapado en errores locales.

* **Pesos de clase**: Como hay pocos edificios con calificación 'A' o 'B', usamos pesos_clases para que la IA les preste más atención y no los ignore.

---


In [ ]:
# --- PASO 1: Define los hiperparámetros (¡Experimenta aquí!) para mejorar el Accuracy ---
capas_ocultas = (10, 20)
iteraciones = 20
tasa_aprendizaje = 0.0005
pesos_clases = {0: 1.63, 1: 2.91, 2: 0.7, 3: 4.05, 4: 10.3, 5: 44.3, 6: 142.2}

# --- PASO 2: Crear el modelo ---
mlp = MLPClassifier(
    hidden_layer_sizes=capas_ocultas,
    activation='relu',
    solver='adam',
    max_iter=iteraciones,
    learning_rate_init=tasa_aprendizaje,
    early_stopping=True,
    validation_fraction=0.1,
    random_state=1234,
    verbose=True # Para ver la evolución del error en cada paso
)

# --- PASO 3: Entrenar ---
print("Entrenando... Observa cómo disminuye el 'loss' (error) en cada iteración.")
mlp.fit(x_train, y_train)

---

### S6.T2. Evaluación del rendimiento del modelo

Finalizado el entrenamiento, es imperativo realizar un diagnóstico del comportamiento del modelo. No se debe considerar únicamente el valor de precisión final, sino también la dinámica del aprendizaje:

* **Análisis de convergencia (Loss y Accuracy):**

>* Función de Pérdida (Loss): Representa el error del modelo. Se busca una disminución monótona y suave. Una curva errática puede indicar una tasa de aprendizaje demasiado elevada.

>* Precisión en Validación (Accuracy): Monitorizamos cómo mejora el acierto en datos no utilizados para el entrenamiento. Si la pérdida disminuye pero la precisión en validación se estanca o empeora, el modelo podría estar incurriendo en sobreajuste (overfitting).

* **Matriz de Confusión Normalizada**: Este análisis permite identificar errores sistemáticos entre categorías adyacentes (por ejemplo, confusión entre las calificaciones B y C).

>* Los valores de la diagonal principal representan el Recuperación (Recall) o sensibilidad por clase: la probabilidad de que el modelo clasifique correctamente un inmueble dada su categoría real.
>* Un valor de 0.85 en la diagonal de la "Clase E" indica que el modelo identifica correctamente el 85% de los edificios de esa categoría.
>* Los valores fuera de la diagonal indican errores sistemáticos. Por ejemplo, un 0.10 en la intersección de Real: B y Predicho: C significa que el 10% de los edificios tipo B son erróneamente clasificados como C.

---

In [ ]:
# --- 1. EVALUACIÓN DE LA DINÁMICA DE APRENDIZAJE ---
plt.figure(figsize=(15, 5))

# Análisis de la Función de Pérdida (Convergencia)
plt.subplot(1, 2, 1)
plt.plot(mlp.loss_curve_, color='#e74c3c', lw=2, label='Training Loss')
plt.fill_between(range(len(mlp.loss_curve_)), mlp.loss_curve_, color='#e74c3c', alpha=0.1)
plt.title('Curva de Convergencia: Función de Pérdida', fontsize=13)
plt.xlabel('Iteraciones (Épocas)')
plt.ylabel('Loss')
plt.grid(True, ls='--', alpha=0.5)
plt.legend()

# Análisis de la Evolución de la Precisión
plt.subplot(1, 2, 2)
plt.plot(mlp.validation_scores_, color='#2ecc71', lw=2, label='Validation Accuracy')
plt.title('Evolución de la Precisión durante el Entrenamiento', fontsize=13)
plt.xlabel('Épocas')
plt.ylabel('Accuracy')
plt.grid(True, ls='--', alpha=0.5)
plt.legend()

plt.tight_layout()
plt.show()

# --- 2. EXAMEN FINAL DEL MODELO (SET DE TEST) ---
y_pred = mlp.predict(x_test)

print("\n" + "="*65)
print("             INFORME TÉCNICO DE MÉTRICAS DE CLASIFICACIÓN")
print("="*65)
# El reporte permite evaluar el rendimiento específico para cada letra energética
print(classification_report(y_test, y_pred, target_names=["G", "F", "E", "D", "C", "B", "A"], zero_division=0))

# Matriz de Confusión Normalizada (Probabilidad de acierto por categoría)
plt.figure(figsize=(8, 6))
cm_norm = confusion_matrix(y_test, y_pred, normalize='true')
sns.heatmap(cm_norm, annot=True, fmt='.2f', cmap='YlGnBu',
            xticklabels=["G", "F", "E", "D", "C", "B", "A"],
            yticklabels=["G", "F", "E", "D", "C", "B", "A"])
plt.title('Análisis de Errores: Matriz de Confusión Normalizada', fontsize=14)
plt.xlabel('Predicción del Modelo')
plt.ylabel('Categoría Real')
plt.show()

---

### S6.T3. Análisis de interpretabilidad mediante SHAP

Las redes neuronales son consideradas a menudo modelos de "caja negra" (black box) debido a la dificultad de trazar cómo los pesos y las funciones de activación transforman los inputs en una predicción. Para resolver esto, utilizaremos los valores SHAP (Shapley Additive Explanations).

¿Cómo interpretar el gráfico de SHAP (Summary Plot)?

* Distribución de importancia: Las variables aparecen ordenadas de arriba a abajo según su influencia global en el modelo. La variable superior es la que más "pesa" en la decisión de la red.

* Impacto (Eje X): Los puntos a la derecha del centro (valor 0) indican que esa variable aumenta la probabilidad de que el edificio pertenezca a la clase analizada (ej. Clase A). Los puntos a la izquierda indican que la disminuyen.

* Valor de la variable (Color): * El rojo representa un valor alto de la característica (ej. muchos metros cuadrados).

* El azul representa un valor bajo (ej. pocos metros cuadrados).

Validación del conocimiento experto: El estudiante debe analizar si la IA ha aprendido relaciones lógicas. Por ejemplo: ¿Es coherente que un valor bajo en biomasa (azul) desplace la predicción hacia la derecha (Clase A)? Si la respuesta es afirmativa, el modelo es físicamente consistente.

---

In [ ]:
# Selección de muestras representativas para reducir el coste computacional
x_test_resumido = shap.sample(x_test, 50)
x_train_resumido = shap.sample(x_train, 100)

print("Ejecutando el cálculo de valores SHAP para la interpretación del modelo...")
explainer = shap.KernelExplainer(mlp.predict_proba, x_train_resumido)
shap_values = explainer.shap_values(x_test_resumido)

# Visualización de la importancia de variables para la Calificación 'A' (Índice 6)
shap_values_array = np.array(shap_values).transpose(2, 0, 1)
print("\nDiagrama de impacto de variables para la Clase A:")
shap.summary_plot(shap_values_array[6], x_test_resumido, plot_type="dot")

# Nota: Evalúa todas las clases cambiando el índice. Por ejemplo, para la Clase G, cambie el índice a [0] según la codificación realizada en la ST5.